In [14]:
import zipfile
import pandas as pd
import io

# Path to the ZIP file
zip_file_path = 'data_YourNextPurchase.zip'

# Dictionary to store DataFrames
dataframes = {}

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # List all files in the ZIP
    file_names = zip_ref.namelist()
    
    # Load CSV files into DataFrames
    for file_name in file_names:
        if file_name.endswith('.csv'):
            with zip_ref.open(file_name) as file:
                # Read the CSV file into a DataFrame
                df_name = file_name.split('/')[-1].split('.')[0]  # Use the file name (without extension) as key
                dataframes[df_name] = pd.read_csv(io.TextIOWrapper(file, encoding='utf-8'))


In [15]:
for name, df in dataframes.items():
    print(f"\nDataframe: {name}")
    print("-" * 50)
    display(df)


Dataframe: clients
--------------------------------------------------


,ClientID,ClientSegment,ClientCountry,ClientOptINEmail,ClientOptINPhone,ClientGender,Age
0,4508698145640552159,LOYAL,USA,1,1,M,NaN
1,2022746661324934183,INACTIVE_1Y,USA,0,1,F,NaN
2,5794452591674300222,LOYAL,USA,1,1,F,NaN
3,678556389231830160,LOYAL,USA,1,1,M,NaN
4,877301557964624234,LOYAL,USA,1,1,F,NaN
...,...,...,...,...,...,...,...
424032,6368205759336936904,LOYAL,USA,1,1,F,NaN
424033,6789036627132562875,LOYAL,USA,1,0,NaN,NaN
424034,6482790535813826054,LOYAL,USA,1,1,F,45.0
424035,6068605136251037073,LOYAL,USA,1,1,NaN,NaN



Dataframe: products
--------------------------------------------------


,ProductID,Category,FamilyLevel1,FamilyLevel2,Universe
0,8588329712199129922,Basketball,Shoes,Adidas Harden Vol. 4,Women
1,2134543135517272693,Football,Shoes,Nike Mercurial Vapor,Women
2,7624925797167026532,Football,Ball,Nike Ordem V,Women
3,2270862675779333216,Football,Jersey,Adidas Home Jersey,Men
4,6504730903144736800,Football,Shorts,Adidas Squadra 21,Women
...,...,...,...,...,...
3148077,3001474681171769417,Softball,Ball,Worth Dream Seam,Men
3148078,5431491360807494905,Softball,Bat,DeMarini CF,Women
3148079,723882298635000031,Softball,Ball,Worth Dream Seam,Men
3148080,559354893668910273,Softball,Bat,DeMarini CF,Men



Dataframe: stocks
--------------------------------------------------


,StoreCountry,ProductID,Quantity
0,AUS,1284651161701379667,2.0
1,AUS,6076274819885027797,2.0
2,AUS,6019386668821120661,2.0
3,AUS,2122575437123245322,2.0
4,AUS,5901681811213086415,2.0
...,...,...,...
16019,ARE,3935326559397911351,9.0
16020,DEU,3935326559397911351,9.0
16021,DEU,6153039859697012898,10.0
16022,ARE,6153039859697012898,10.0



Dataframe: stores
--------------------------------------------------


,StoreID,StoreCountry
0,7793989562498223120,BRA
1,5301108587679203543,BRA
2,2908896877692945358,BRA
3,2718131790848668199,BRA
4,7819332417840564556,BRA
...,...,...
601,4492541004792360162,USA
602,8606906800192283162,USA
603,5478106795462651143,USA
604,5524089218753938698,USA



Dataframe: transactions
--------------------------------------------------


,ClientID,ProductID,SaleTransactionDate,StoreID,Quantity,SalesNetAmountEuro
0,5159341040532191492,7696661559050891210,2024-06-16 00:00:00+00:00,5202366601222773915,4,32.97
1,8169347797101088046,1086147279819320560,2024-12-10 00:00:00+00:00,5906069635056395270,3,87.98
2,8119209481417068505,3532473209579560668,2023-06-06 00:00:00+00:00,4821951108133690356,4,56.97
3,6144661105431601127,2935450247977233959,2024-04-04 00:00:00+00:00,7042378510114251866,2,17.99
4,2497726585282787281,5103640511191568912,2023-09-20 00:00:00+00:00,1450109522794525790,1,5.99
...,...,...,...,...,...,...
2608189,1003100987571803008,242506609152720728,2024-09-25 00:00:00+00:00,9099531021898466037,1,29.99
2608190,5201324372064592752,412909163137289630,2025-01-22 00:00:00+00:00,7913334853958149044,2,29.99
2608191,7078829430266590284,2568137681619865644,2024-11-21 00:00:00+00:00,5202366601222773915,3,17.98
2608192,2660995687537176804,219531566035948655,2024-11-30 00:00:00+00:00,7072849857188684803,1,5.99


In [16]:
# Extract individual dataframes from the dictionary
clients_df = dataframes['clients']
products_df = dataframes['products']
stocks_df = dataframes['stocks']
stores_df = dataframes['stores']
transactions_df = dataframes['transactions']

# Print basic information about each dataframe
for df_name, df in [('Clients', clients_df), 
                    ('Products', products_df),
                    ('Stocks', stocks_df),
                    ('Stores', stores_df),
                    ('Transactions', transactions_df)]:
    print(f"\n{df_name} DataFrame Shape: {df.shape}")


Clients DataFrame Shape: (424037, 7)

Products DataFrame Shape: (3148082, 5)

Stocks DataFrame Shape: (16024, 3)

Stores DataFrame Shape: (606, 2)

Transactions DataFrame Shape: (2608194, 6)
